In [1]:
import pandas as pd
import jieba
import re
import cn2an

In [2]:
df = pd.read_csv('./icook_sort.csv')

In [3]:
df.head()

,_id,RecipeID,CategoryName,SubCategoryID,SubCategoryName,RecipeName,Description,AuthorID,Author,Servings,CookingTime,Ingredient,CookingSteps,LikeStat,Datetime,Preview,URL,Ingredient_Pro
0,60646cc365cd68e35c01e814,170303,家常菜色,"['104', '602']","['烤肉', '中秋料理']",綜合雞肉串燒,用平底鍋或烤箱就能完成的雞肉串燒，其中照燒烤肉醬只要幾種材料組合，不必擔心市售烤肉醬有過多添...,100002279834418,胖仙女,4人份,45分鐘,"{'去骨雞腿': '4片', '起士明太子醬': '適量', '蔥白': '適量', '檸檬...",['1. 雞腿肉切小塊的同時，切去皮及多餘的脂肪，取其中2/3的雞肉，加入所有的醃料拌勻，醃...,662,2016-09-11,2.4 萬 瀏覽,https://icook.tw/recipes/170303,"{'去骨雞腿': ['4', '片'], '起士明太子醬': ['1', '適量'], '蔥..."
1,60646cc365cd68e35c01e815,169048,家常菜色,"['104', '77']","['烤肉', '宵夜']",燒烤味增秋葵透抽捲,秋葵正當季，適逢中秋節（烤肉節）腳步漸漸接近，來道『烤味增秋葵透抽』吧！\n簡單備料，有彈性...,charminglydia,懶廚房,2人份,20分鐘,"{'透抽': '1尾', '黃秋葵': '6個', '味增燒烤醬': '適量', '白芝麻'...","['1. 食材大集合。', '2. 黃秋葵先在鹽上滾一滾，如此讓表面絨毛軟化，較好入口。',...",675,2016-09-01,1.7 萬 瀏覽,https://icook.tw/recipes/169048,"{'透抽': ['1', '尾'], '黃秋葵': ['6', '個'], '味增燒烤醬':..."
2,60646cc365cd68e35c01e816,155271,家常菜色,"['104', '77', '602']","['烤肉', '宵夜', '中秋料理']",沙嗲烤雞肉串,這個充滿東南亞風味的沙嗲烤雞串，用沙嗲醬醃雞肉，然後放入烤箱烤，這可令雞肉更嫰滑多汁。再加上...,MrsPsKitchen,Mrs P's Kitchen,4人份,45分鐘,"{'雞肉': '600克', '沙嗲醬': '1大匙', '醬油': '1小匙', '糖':...","['1. 將雞肉切小塊，然後用醃料醃雞肉最少一個小時', '2. 將竹籤浸在熱水5分鐘，這可...","1,669",2016-06-02,3.3 萬 瀏覽,https://icook.tw/recipes/155271,"{'雞肉': ['600', '克'], '沙嗲醬': ['1', '大匙'], '醬油':..."
3,60646cc365cd68e35c01e817,90086,家常菜色,"['104', '602']","['烤肉', '中秋料理']",甜柿烤肉串,霜降吃柿子！甜柿營養豐富，卻不適合空腹吃。與肉類搭配，解油膩又清爽。簡單做法，最適合週末小派對！,jj5colorkitchen,jj5色廚,3人份,30分鐘,"{'火鍋白肉片': '6-7 片', '甜柿': '1 顆', '百里香': '適量', '...",['1. 這種豬肉片就是平常酸菜白肉鍋用的白肉片，等於沒薰過的培根肉，形狀規則，油脂豐富，比...,671,2014-10-25,1.8 萬 瀏覽,https://icook.tw/recipes/90086,"{'火鍋白肉片': ['6-7', '片'], '甜柿': ['1', '顆'], '百里香..."
4,60646cc365cd68e35c01e818,337797,家常菜色,"['104', '61']","['烤肉', '韓式']",韓式烤肉飯,百吃不膩的韓式烤肉飯，保證讓人胃口大開！無論配麵、配飯都很搭。 作法超級簡單！適合料理初學者...,1f2cb7b4bee5e7d2,幸福餐桌,2人份,45分鐘,"{'沙朗牛肉片': '400克', '蒜': '3瓣', '洋蔥': '1顆', '蘋果':...","['1. 以刀背將牛肉片敲打鬆軟', '2. 牛肉片切成小塊狀', '3. 將蒜、奇異果、蘋...",53,2020-05-23,"8,026 瀏覽",https://icook.tw/recipes/337797,"{'沙朗牛肉片': ['400', '克'], '蒜': ['3', '瓣'], '洋蔥':..."


In [4]:
# 去除個欄位空值資料
df_word2vec = df.dropna()

In [5]:
# 匯入大量詞庫字典
jieba.set_dictionary('jieba_data/dict.txt')

In [6]:
def strQ2B(s):
    """把字串全形轉半形"""
    rstring = ""
    for uchar in s:
        u_code = ord(uchar)
        if u_code == 12288:  # 全形空格直接轉換
            u_code = 32
        elif 65281 <= u_code <= 65374:  # 全形字元（除空格）根據關係轉化
            u_code -= 65248
        rstring += chr(u_code)
    return rstring

In [7]:
# 匯入停用字典
with open('./jieba_data/chinese_sw.txt', 'r', encoding='utf-8') as f:
    stop_words_list = f.read().split('\n')
stop_words_list[:10]

['啊', '阿', '哎', '哎呀', '哎喲', '唉', '俺', '有點', '一直', '俺們']

In [8]:
#針對Ingredient 做清洗

for Ingredients in df_word2vec['Ingredient']:
    tmpList = []
    for Ingredient in eval(Ingredients).keys():
        
        # 將文字全形轉半形
        Ingredient = strQ2B(Ingredient)
        
        # 針對文字使用正規表達式做整理 (第一次)
        Ingredient = re.sub(r'(\(.*\))', '', Ingredient) # 有括號內的文字刪除
        Ingredient = re.sub(r'(【.*】)', '', Ingredient) # 有括號內的文字刪除
        Ingredient = re.sub(r'[\W+]', '', Ingredient)  # 將非文數字的符號刪除
        Ingredient = re.sub(r'[\W+]', '', Ingredient)  # 將非文數字的符號刪除
        Ingredient = re.sub(r'[a-zA-Z]', '', Ingredient) # 將英文字刪除
        Ingredient = re.sub(r'[1-9]', '', Ingredient) # 將數字刪除
        tmpList.append(Ingredient)
    WordString = ' '.join(tmpList)
    with open('./word2vec_data/icook_Ingredient_seg.txt', 'a', encoding='utf-8') as f:
        f.write(WordString + '\n')
        




In [9]:
# 針對 CookingStep 做清洗

for CookingSteps in df_word2vec['CookingSteps']:
    tmpList = []
    for EachStep in eval(CookingSteps):  # 原字串類別轉回字典
        
        # 將文字全形轉半形
        EachStep = strQ2B(EachStep)
        
        # 針對文字使用正規表達式做整理 (第一次)
        EachStep = re.sub(r'(^\d. )', '', EachStep) # 將步驟 1. 2. 3. ....前面的數字順序刪除
        EachStep = re.sub(r'(\(.*\))', '', EachStep) # 有括號內的文字刪除
        
        # 將阿拉伯數字轉換成漢字數字 (EX. 4 => 四)
        EachStep = cn2an.transform(EachStep, "an2cn")
        
        # 針對文字使用正規表達式做整理 (第二次)
        EachStep = re.sub(r'[\W+]', '', EachStep)  # 將非文數字的符號刪除
        EachStep = re.sub(r'[a-zA-Z]', '', EachStep) # 將英文字刪除
        
        # Jieba 斷詞
        EachStepWords = jieba.lcut(EachStep)
        
        # 對每個斷詞做停用字的篩選
        UserfulWords = []       
        for EachStepWord in EachStepWords:
            if EachStepWord not in stop_words_list:
                UserfulWords.append(EachStepWord)

        EachStepWordsString = ' '.join(UserfulWords) # 每個食譜每個步驟所段的詞轉化成字串
        tmpList.append(EachStepWordsString)          # 每個食譜每個步驟所段的詞放入List
    StepsWordString = ' '.join(tmpList)              # 將每個食譜所有步驟的 List 轉化成字串
    with open('./word2vec_data/icook_CookingStep_seg.txt', 'a', encoding='utf-8') as f:
        f.write(StepsWordString + '\n')
   
     

Building prefix dict from C:\Users\CEB10201\PycharmProjects\Python_Recipe_ETL\jieba_data\dict.txt ...
Loading model from cache C:\Users\CEB10201\AppData\Local\Temp\jieba.u8d2bce68ea74a61e4827369d82e5dd51.cache
Loading model cost 0.560 seconds.
Prefix dict has been built successfully.


WARN: 输入的数据不在转化范围内：٩！
WARN: 输入的数据不在转化范围内：٩！
WARN: 输入的数据不在转化范围内：٩！
WARN: 输入的数据不在转化范围内：٩！
WARN: 超出数据范围，最长支持 16 位
WARN: 输入的数据不在转化范围内：๑！
WARN: 输入的数据不在转化范围内：๑！
WARN: 输入的数据不在转化范围内：٩！


In [10]:
# 針對 Description 做清洗

for Description in df_word2vec['Description']:

    # 將文字全形轉半形
    Description = strQ2B(Description)

    # 針對文字使用正規表達式做整理 (第一次)
    Description = re.sub(r'(\(.*\))', '', Description) # 有括號內的文字刪除

    # 將阿拉伯數字轉換成漢字數字 (EX. 4 => 四)
    Description = cn2an.transform(Description, "an2cn")

    # 針對文字使用正規表達式做整理 (第二次)
    Description = re.sub(r'[\W+]', '', Description)    # 將非文數字的符號刪除
    Description = re.sub(r'[a-zA-Z]', '', Description) # 將英文字刪除

    # Jieba 斷詞
    WordsList = jieba.lcut(Description)
    
    # 對每個斷詞做停用字的篩選
    UserfulWords = []
    for EachWord in WordsList:
        if EachWord not in stop_words_list:
            UserfulWords.append(EachWord)
            
    WordsString = ' '.join(UserfulWords)      
    with open('./word2vec_data/icook_Description_seg.txt', 'a', encoding='utf-8') as f:
        f.write(WordsString + '\n')

       


WARN: 输入的数据不在转化范围内：٩！
WARN: 输入的数据不在转化范围内：٩！
WARN: 输入的数据不在转化范围内：٩！
WARN: 输入的数据不在转化范围内：٩！
WARN: 输入的数据不在转化范围内：٩！
WARN: 输入的数据不在转化范围内：۶！
WARN: 输入的数据不在转化范围内：٩！
WARN: 输入的数据不在转化范围内：٩！
WARN: 输入的数据不在转化范围内：๑！
WARN: 输入的数据不在转化范围内：๑！
WARN: 输入的数据不在转化范围内：٩！
WARN: 输入的数据不在转化范围内：๑！
WARN: 输入的数据不在转化范围内：੭！
WARN: 输入的数据不在转化范围内：٩！
WARN: 输入的数据不在转化范围内：٩！


In [1]:
from gensim.models import word2vec
import time

start_time = time.time()

# Settings
seed = 1                # seed： 亂數種子
sg = 1                  # sg： Word2Vec 有兩種算法，CBOW 以及 Skip-gram
window_size = 3        # window_size： 周圍詞彙要看多少範圍
vector_size = 20        # size： 轉成向量的維度
min_count = 10           # min_count： 詞頻少於 min_count 之詞彙不會參與訓練
workers = 8             # workers： 訓練的並行數量
epochs = 100             # iter： 訓練的迭代次數
batch_words = 10000   # batch_words：每次給予多少詞彙量訓練


# 訓練資料
train_data = word2vec.LineSentence('./word2vec_data/icook_CookingStep_seg.txt')    # CookingStep
# train_data = word2vec.LineSentence('./word2vec_data/icook_Ingredient_seg.txt')  # Ingredient
# train_data = word2vec.LineSentence('./word2vec_data/icook_Description_seg.txt')    # Description


 

model = word2vec.Word2Vec(
    train_data,
    min_count=min_count,
    vector_size=vector_size,
    workers=workers,
    epochs=epochs,
    window=window_size,
    sg=sg,
    seed=seed,
    batch_words=batch_words
)

end_time = time.time()
print('總共花費 {} 秒'.format(end_time - start_time))

c:\users\ceb10201\pycharmprojects\python_recipe_etl\venv\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


總共花費 250.09647154808044 秒


In [2]:
model

In [3]:
model.corpus_total_words

2019766

In [7]:
model.wv.most_similar("番茄", topn=10)

[('蕃茄', 0.9696077704429626),
 ('聖女', 0.9029524326324463),
 ('洋菇', 0.9028424024581909),
 ('磨菇', 0.9012870788574219),
 ('甜椒', 0.8999719619750977),
 ('蘑菇', 0.8932895660400391),
 ('西洋', 0.8720659017562866),
 ('芹', 0.8705599904060364),
 ('彩椒', 0.8661943078041077),
 ('紅椒', 0.8541717529296875)]

In [5]:
# # Model 儲存CookingStep
# model.save('./word2vec_data/icook_CookingStep_word2vec.model')

# # Model 儲存 Description
# model.save('./word2vec_data/icook_Description_word2vec.model')

# Model 儲存 Ingredient
model.save('./word2vec_data/icook_Ingredient_word2vec.model')